## Сбор данных

*Подключение библиотек для сбора данных*

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

*Создание списка с городами, по которым наш парсер будет проходиться*

In [ ]:
CITY = [
    "moskva", "sankt-peterburg", "vladimir", "volgograd", "voronezh", "ekaterinburg",
    "ivanovo", "kazan", "kaluga", "kostroma", "krasnodar", "krasnoyarsk",
    "nizhniy_novgorod", "novosibirsk", "omsk", "perm", "rostov-na-donu", "samara",
    "saratov", "tver", "tula", "ufa", "chelyabinsk", "yaroslavl"
]

*Генерируем URL для страницы с объявлениями об автомобилях по указанному городу и номеру страницы*

In [ ]:
def get_url(city, page):
    return f"https://auto.ru/{city}/cars/used/?page={page}"

*Получаем количество страниц с объявлениями об автомобилях в конкретном городе*

In [ ]:
def get_pagenation(driver):
    return int(driver.find_element(by = By.XPATH, value = "//a[contains(@class, 'ListingPagination__page')][last()]").get_attribute("href").split("page=")[1])

*Извлекаем URL-адреса объявлений об автомобилях с текущей страницы и записываем их в список*

In [ ]:
def get_url_from_page(driver):
    array_auto = driver.find_elements(by = By.XPATH, value = "//a[@class='Link ListingItemTitle__link']")
    array_url_auto = []
    for i in range(len(array_auto)):
        array_url_auto.append(array_auto[i].get_attribute("href"))
    return array_url_auto

*Отключаем обнаружение WebDriver, создаём экземпляр веб-драйвера Firefox и запускаем наш парсер*

In [ ]:
options = webdriver.FirefoxOptions()
options.set_preference("dom.webdriver.enabled", False)

driver = webdriver.Firefox(options=options)

driver.close()